In [ ]:
text = """
### Instruction: 

You are given a biomedical sentence. 
Your task is to determine whether it explicitly expresses generalizable factual knowledge that can be structured as a triple, consisting of a subject, a relation, and an object.

Such knowledge includes, but is not limited to:
- Biomedical facts
- Biological or chemical mechanisms
- Causal or regulatory relationships
- Descriptions of widely applicable clinical or procedural techniques

You should not mark a sentence as factual if it only:
- Describes specific case observations
- States intentions to study something
- Mentions experimental setups without reporting results
- Provides background information without asserting a general fact

### Output Format:
{schema_instruction}

### Input: In this study, we aimed to evaluate the effects of a new drug on blood pressure in hypertensive rats.
### Output:
```json
{{
    "factuality": false,
    "triple": null,
    "reason": "This sentence only states the study’s intention without reporting any factual findings or generalizable knowledge. It does not describe a subject-relation-object fact but rather the experimental purpose."
}}
```

### Input: Vitamin D supplementation reduces the risk of osteoporosis in elderly patients.
### Output:
```json
{{
    "factuality": true,
    "triple": {{
        "subject": "Vitamin D supplementation",
        "relation": "reduces risk of", 
        "object": "osteoporosis in elderly patients"
    }},
    "reason": "This sentence expresses a general biomedical fact that can be structured as a triple: (Vitamin D supplementation) – (reduces risk of) – (osteoporosis in elderly patients). It asserts a clear causal relationship that is generalizable and factual."
}}
```
"""

In [ ]:
### Version 1 of the adaxeval generation prompt
text = """\
### Instruction: 
Given a biomedical sentence and its associated knowledge triple, follow the steps below to generate a multiple-choice question:

1. Generate a Fill-in-the-Blank Prompt: Create a fill-in-the-blank prompt for the triple by using the subject and relation as context, placing the object at the end as [BLANK]. Do not largely modify the original sentence, but ensure it is grammatically correct and clear. The prompt should be a complete sentence that can stand alone.
2. Generate Distractors: Create three plausible but incorrect choices (distractors). Each distractor should:
    - Be similar in length and semantic category to the given object, please ensure they are not too short or too long compared to the object.
    - Be relevant to the subject and relation, but not the correct answer.
3. Paraphrase into a Question: Rephrase the fill-in-the-blank prompt into a well-formed, self-contained question that asks for the object.
    - Use diverse vocabulary and a different linguistic style or structure than the original sentence. Avoid simply rearranging the original
    - Do not introduce any new factual information or semantic categories that are not explicitly present in the original triple and may lead to guidance to guess the answer from candidates.
5. Format the Output: Structure the output as a JSON object containing the prompt, question, and distractors.

### Input:     
```json
{
    "sentences": "Seven distinct glycosidases (EC 3.2) have been characterized in guinea-pig epidermis.",
    "triple": {
        "subject": "guinea-pig epidermis",
        "relation": "characterized",
        "object": "seven distinct glycosidases"
    }
}
```

### Output:
```json
{
    "prompt": "[BLANK] have been characterized in guinea-pig epidermis.",
    "distractors": [
        "five lysosomal enzymes",
        "high-activity proteases",
        "low-activity amylases"
    ],
    "question": "What substances have researchers discovered within the skin tissue of guinea pigs?"
}
```

### Input:     
```json
{
    "sentences": "Primary treatment of malignant melanoma is a wide range excision of the tumor lesion and its surrounding normal tissue.",
    "triple": {
        "subject": "Primary treatment of malignant melanoma",
        "relation": "is",
        "object": "wide range excision of the tumor lesion and its surrounding normal tissue"
    }
}
```

### Output:
```json
{
    "prompt": "Primary treatment of malignant melanoma is [BLANK].",
    "distractors": [
        "localized removal of cancerous skin and nearby healthy cells",
        "focused radiation of the melanoma site and adjacent tissues",
        "targeted excision of affected area and peripheral skin margins"
    ],
    "question": "What medical procedure is primarily used to treat malignant melanoma at its initial stage?"
}
```
"""

import json
escaped_text = json.dumps({"system": text}, ensure_ascii=False, indent=4)
print(escaped_text)

{
    "system": "### Instruction: \nGiven a biomedical sentence and its associated knowledge triple, follow the steps below to generate a multiple-choice question:\n\n1. Generate a Fill-in-the-Blank Prompt: Create a fill-in-the-blank prompt for the triple by using the subject and relation as context, placing the object at the end as [BLANK]. Do not largely modify the original sentence, but ensure it is grammatically correct and clear. The prompt should be a complete sentence that can stand alone.\n2. Generate Distractors: Create three plausible but incorrect choices (distractors). Each distractor should:\n    - Be similar in length and semantic category to the given object, please ensure they are not too short or too long compared to the object.\n    - Be relevant to the subject and relation, but not the correct answer.\n3. Paraphrase into a Question: Rephrase the fill-in-the-blank prompt into a well-formed, self-contained question that asks for the object.\n    - Use diverse vocabular

In [3]:
### Version 2 of the adaxeval generation prompt

text = """\
### Instruction: 

You are given
- A biomedical sentence that expresses one or more facts.
- A reference triple (subject, relation, object) derived from the sentence.

Your goal is to produce a high-quality multiple-choice question that checks understanding of the key fact in the sentence.
Your output must include:
- A triple (improved if necessary)
- A fill-in-the-blank prompt
- The correct answer
- Three distractors
- A paraphrased question

#### Step 1: Generate / Validate the Factual Knowledge Triple
Generate a factual knowledge triple: Given the original sentence containing specific factual knowledge and a reference of the triple, generate a new factual triple that can accurately captures the key factual information from the sentence. 
- The newly generated triple can be the same as the reference triple if it is already high-quality, or it can be a modified version that better meets the criteria below.
The triple must be:
1. **Factually faithful**: Matches the meaning of the original sentence (no additions, no omissions).
2. **Specific & self-contained**:
    - Subject and object must include necessary qualifiers (e.g., disease name, anatomical location).
    - Avoid vague placeholders (“these techniques,” “this process,” “it”).
    - Someone reading only the triple should know what the fact means without needing the sentence.
3. **Relation clarity**: Use a meaningful, grammatical relation (e.g., “is characterized by,” “causes,” “consists of”).

#### Step 2: Create a Fill-in-the-Blank Prompt
1. **Choose the blanked part intelligently**: 
    - Normally, you will blank out the object, but if blanking the object creates an awkward, unclear, or non-noun answer (e.g., “these techniques,” vague pronouns, or non-specific phrases), you may instead blank the subject.
    - The goal is to make the [BLANK] a specific, self-contained noun phrase that is a clear, correct answer option.
2. **Start from the original sentence:**: 
    - Use the sentence’s wording and structure as your base. 
    - Do not rewrite or paraphrase except when necessary to insert [BLANK] or fix grammar.
3. **Replace only the subject or object:**: 
    Substitute the subject or object phrase with [BLANK] exactly where it appears. Keep all other terms, qualifiers, and ordering unchanged.
4. **Preserve completeness and context:**: 
    Keep the sentence fully grammatical and self-contained after blanking (e.g., retain disease names, anatomical sites, conditions, or qualifiers present in the original). Don’t oversimplify.
5. **No distortion**: 
    Do not add new information or change the factual meaning. The blanked version must remain faithful to the original statement.

#### Step 3: Generate the Answer
- Answer = the exact phrase you blanked out (subject or object).
- Copy it exactly from the improved triple if possible, or adapt minimally to match the blank in your prompt.
- This will be the correct multiple-choice answer.

#### Step 4: Generate Three Distractors
Each distractor must:
1. **Be plausible but incorrect**: fits the subject and relation but not the true fact.
2. Match the semantic type and length of the correct object.
    - If the correct object is long (e.g., a multi-part list), you may modify it by distorting one or two key elements while preserving the same overall format and length.
    - Apply the same principle to distractors: create parallel, similarly structured lists of similar length.
    - Do not make the correct answer long and the distractors much shorter or vice versa; keep them comparable.
3. **Remain biomedical and credible**: Do not invent absurd or trivial options.
4. Avoid being obviously wrong, too generic, or absurd.
5. Avoid trivial negations (“not X”).

Use biomedical terms from the same domain when possible to make them challenging but fair.

#### Step 4: Paraphrase into a Question
1. Rewrite the fill-in-the-blank prompt into a well-formed, self-contained question asking for the object.
2. Change the sentence structure and vocabulary; do not just move words around.
3. Do not add or drop factual information or hints.
4. The question should still be answerable solely by the fact expressed in the triple.

### Format of Input:
```json
{
    "sentence": <original sentence>,
    "triple": {
        "subject": <subject>,
        "relation": <relation>,
        "object": <object>
    }
}
```

### Format of Output:
```json
{
    "triple": {
        "subject": "<subject>",
        "relation": "<relation>",
        "object": "<object>"
    },
    "fill_in_blank": "<fill-in-the-blank sentence with [BLANK]>",
    "question": "<paraphrased question>",
    "answer": "<subject or object>",
    "distractors": [
        "<distractor 1>",
        "<distractor 2>",
        "<distractor 3>"
    ]
}
```

### Input:
```json
{
    "sentence": "Seven distinct glycosidases (EC 3.2) have been characterized in guinea-pig epidermis.",
    "triple": {
        "subject": "guinea-pig epidermis",
        "relation": "characterized",
        "object": "seven distinct glycosidases"
    }
}
```

### Output:
```json
{
    "triple": {
        "subject": "guinea-pig epidermis",
        "relation": "has been characterized to contain",
        "object": "seven distinct glycosidases (EC 3.2)"
    },
    "fill_in_blank": "[BLANK] have been characterized in guinea-pig epidermis.",
    "question": "How many distinct glycosidases (EC 3.2) have been identified in the epidermis of the guinea pig?",
    "answer": "seven distinct glycosidases (EC 3.2)",
    "distractors": [
        "five distinct glycosidases (EC 3.2)",
        "nine distinct glycosidases (EC 3.2)",
        "twelve distinct glycosidases (EC 3.2)"
    ]
}
```

### Input:     
```json
{
    "sentence": "In mammals, 13 AQPs isoforms have been identified, and they are expressed in a variety of tissues, including the kidney, the brain, the eye and the respiratory tract.",
    "triple": {
        "subject": "AQPs",
        "relation": "are expressed in",
        "object": "a variety of tissues including the kidney, the brain, the eye, and the respiratory tract"
    }
}
```

### Output:
```json
{
    "triple": {
        "subject": "AQPs isoforms in mammals",
        "relation": "are expressed in a variety of tissues, including",
        "object": "the kidney, the brain, the eye and the respiratory tract"
    },
    "fill_in_blank": "In mammals, 13 AQPs isoforms have been identified, and they are expressed in a variety of tissues, including [BLANK]",
    "question": "Which tissues in mammals express the identified AQP isoforms?",
    "answer": "the kidney, the brain, the eye and the respiratory tract",
    "distractors": [
        "the liver, pancreas, spleen and gallbladder",
        "skeletal muscle, adipose tissue, cartilage and bone marrow",
        "the stomach, small intestine, large intestine and pancreas"
    ]
}
```

### Input:     
```json
{
    "sentence": "These techniques provide a method of embryo cloning since the nuclei of early (and perhaps advanced) stage embryos can be transferred into the ooplast to produce several new individuals of identical genotype.",
    "triple": {
        "subject": "These techniques",
        "relation": "provide a method of",
        "object": "embryo cloning"
    }
}
```

### Output:
{
    "triple": {
        "subject": "transfer of nuclei from early stage embryos into the ooplast",
        "relation": "produce",
        "object": "new individuals of identical genotype"
    },
    "fill_in_blank": "The nuclei of early (and perhaps advanced) stage embryos can be transferred into the ooplast to produce several [BLANK].",
    "question": "What can result from transferring nuclei of early stage embryos into the ooplast?",
    "answer": "new individuals of identical genotype",
    "distractors": [
        "genetically diverse offspring",
        "haploid embryos lacking full genomes",
        "embryos with random mutations"
    ]
}

### Input: 
```json
{
    "sentences": "The early steps of DSB-induced ubiquitylation are catalyzed by the RNF8 and RNF168 E3 ubiquitin ligases, and the E2 ubiquitin conjugating enzyme, UBC13 (UBE2N).",
    "triple": {
        "subject": "RNF8, RNF168, and UBC13 (UBE2N)",
        "relation": "catalyze",
        "object": "the early steps of DSB-induced ubiquitylation"
    }
}
```

### Output: 
```json
{
    "triple": {
        "subject": "the early steps of DSB-induced ubiquitylation",
        "relation": "are catalyzed by",
        "object": "RNF8, RNF168 E3 ubiquitin ligases and UBC13 (UBE2N)"
    },
    "fill_in_blank": "The early steps of DSB-induced ubiquitylation are catalyzed by [BLANK].",
    "question": "Which enzymes are responsible for catalyzing the early stages of DSB-induced ubiquitylation?",
    "answer": "RNF8, RNF168 E3 ubiquitin ligases and UBC13 (UBE2N)",
    "distractors": [
        "BRCA1, BARD1 and RAD51 recombinases",
        "MDM2, HUWE1 and CUL4 ubiquitin ligases",
        "SMC5, SMC6 and NBS1 DNA repair proteins"
    ]
}
```

### Input: 
```json
{
    "sentences": "The effectiveness of epidural analgesics is well known.",
    "triple": {
        "subject": "epidural analgesics",
        "relation": "effectiveness is well known",
        "object": "effectiveness"
    }
}
```

### Output:
```json
{
    "triple": {
        "subject": "epidural analgesics",
        "relation": "are known for their",
        "object": "effectiveness"
    },
    "fill_in_blank": "The [BLANK] of epidural analgesics is well known.",
    "question": "What characteristic of epidural analgesics is widely recognized?",
    "answer": "effectiveness",
    "distractors": [
        "toxicity",
        "addictive potential",
        "anti-inflammatory property"
    ]
}
```
"""

import json
escaped_text = json.dumps({"system": text}, ensure_ascii=False, indent=4)
print(escaped_text)

{
    "system": "### Instruction: \n\nYou are given\n- A biomedical sentence that expresses one or more facts.\n- A reference triple (subject, relation, object) derived from the sentence.\n\nYour goal is to produce a high-quality multiple-choice question that checks understanding of the key fact in the sentence.\nYour output must include:\n- A triple (improved if necessary)\n- A fill-in-the-blank prompt\n- The correct answer\n- Three distractors\n- A paraphrased question\n\n#### Step 1: Generate / Validate the Factual Knowledge Triple\nGenerate a factual knowledge triple: Given the original sentence containing specific factual knowledge and a reference of the triple, generate a new factual triple that can accurately captures the key factual information from the sentence. \n- The newly generated triple can be the same as the reference triple if it is already high-quality, or it can be a modified version that better meets the criteria below.\nThe triple must be:\n1. **Factually faithful*

In [ ]:
## Filtering

text = """\
### Instruction

You are given a JSON input containing:
- A **biomedical sentence**
- An associated **knowledge triple** (`subject`, `relation`, `object`)
- A **fill-in-the-blank query**
- A **question-style paraphrase**

Your task is to evaluate each component and return a JSON output with the specified structure.

#### 1. Fill-in-the-Blank Query Evaluation
A fill-in-the-blank query is **Good** only if **all** conditions hold:
- **Faithful reconstruction:** Inserting the `object` into `[BLANK]` reproduces a sentence that keeps the original **meaning and factual correctness**.
- **Contextual completeness:** 
  - The query must contain enough information to answer correctly from the sentence **without needing extra context**. If key qualifiers are missing and make the question ambiguous, mark Poor.
  - Avoid overly broad or context-dependent claims that require knowing the surrounding text to interpret.
- **Syntactic & semantic clarity:** The prompt must be grammatical and unambiguous when read alone.
- **Factual fidelity:** The blank-completed sentence must remain fully true to the original biomedical statement — no extra assumptions, no meaning drift.
- **Specificity & self-containment:** 
  - Filling the blank must be concrete and informative enough to understand the fact without seeing the original given sentence.
  - Reject vague placeholders like “these techniques,” “this process,” “it,” “they,” or other context-dependent terms.
- Biomedical knowledge requirement: The sentence should express a generalizable biomedical fact
  - Reject sentences that only describe a specific case setup, procedural arrangement, or scenario details without encoding a meaningful, general biomedical fact. 
  - Context-dependent procedural or experimental descriptions are Poor.
- **Strict evaluation**: When in doubt, prefer the lower rating (Poor) to avoid overestimating quality.

#### 2. Question-Style Paraphrasing Evaluation
A paraphrase is Good only if **all** conditions hold:
- **Semantic equivalence**: The question asks for exactly the same fact as the fill-in-the-blank sentence (no scope change or reinterpretation).
- **Self-containment**: 
  - The question must be fully understandable on its own without needing the original sentence. It should contain enough detail (subject, relation, qualifiers) to let a reader know what is being asked.
  - Mark Poor if the fact is under-specified or unclear outside the original sentence.
- **Grammatical and natural:**: Sounds like a real human question; fluent and correct English.
- **Biomedical fact focus**: 
  - Avoid paraphrases that just restate procedural or experimental steps or case scenarios instead of testing biomedical knowledge. 
  - If the underlying sentence doesn’t encode a meaningful biomedical fact, mark the paraphrase Poor.
- **Strict evaluation**: When in doubt, prefer the lower rating (Poor) to avoid overestimating quality.

#### 3. Answer and Distractor Quality Evaluation
An answer and distractors are Good only if **all** conditions hold:
- **Answer correctness:** The answer must exactly match the blanked phrase from the fill-in-the blank.
- **Distractor plausibility:** Each distractor should be a credible, biomedical term or phrase that fits the subject/object and relation but is clearly incorrect.
- **Semantic & length consistency:** Distractors should be similar in length and semantic category to the correct answer.
- **No trivial negations or absurd options:** Avoid obviously wrong, too generic, or absurd distractors.
- **Strict evaluation**: When in doubt, prefer the lower rating (Poor) to avoid overestimating quality.

### Additional Practical Checks
- **Stand-alone interpretability test (for both triple & fill-in-blank):**
  If they cannot confidently restate the intended fact or object because of missing qualifiers or vague references, mark No/Poor.
  - If they cannot confidently restate the intended fact or object because of missing qualifiers or vague references, mark No/Poor.
- **Error bias guard:**
  Prefer No/Poor if there is any ambiguity about whether the fact can stand alone.

### Input Format (JSON)
```json
{
    "sentence": "<biomedical sentence>",
    "triple": {
      "subject": "<subject>",
      "relation": "<relation>",
      "object": "<object>"
    },
    "fill_in_blank": "<fill-in-the-blank query>",
    "paraphrase": "<question-style paraphrase>",
    "answer": "<correct answer>",
    "distractors": ["<distractor 1>", "<distractor 2>", "<distractor 3>"]

}
```

### Output Format (JSON)
```json
{
    "explanations": {
      "fill_in_blank_quality": "<short explanation>",
      "paraphrase_quality": "<short explanation>",
      "options_quality": "<short explanation>"
    },
    "fill_in_blank_quality": "Good | Poor",
    "paraphrase_quality": "Good | Poor",
    "options_quality": "Good | Poor"
}
```

### Input:
```json
{
    "sentence": "In particular, damage can be drastically accelerated by the intervention of water and some chemicals such as surfactants and salts.",
    "triple": {
        "subject": "damage",
        "relation": "can be drastically accelerated by",
        "object": "water and chemicals such as surfactants and salts"
    },
    "fill_in_blank": "In particular, damage can be drastically accelerated by the intervention of [BLANK].",
    "question": "Which substances can significantly speed up the process of damage according to the statement?",
    "answer": "water and chemicals such as surfactants and salts",
    "distractors": [
        "oxygen and chemicals such as antioxidants and vitamins",
        "light and chemicals such as enzymes and hormones",
        "heat and chemicals such as acids and bases"
    ]
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Poor — Although grammatically fine, the subject is just “damage” without specifying what is being damaged. The fact isn’t self-contained: a reader cannot know what type of damage (tissue, material, DNA, etc.) is meant, so the biomedical fact is ambiguous and context-dependent.",
    "paraphrase_quality": "Poor — The question repeats the vagueness of “damage” and thus doesn’t let the reader know what is being damaged. It fails the self-contained interpretability requirement.",
    "options_quality": "Good — The correct answer matches the blank and the distractors are plausible chemical/environmental agents of damage, similar in length and category."
  },
  "fill_in_blank_quality": "Poor",
  "paraphrase_quality": "Poor",
  "options_quality": "Good"
}
```

### Input:
```json
{
  "sentence": "Carcinoma rarely occurs at a colostomy site.",
  "triple": {
    "subject": "carcinoma",
    "relation": "rarely occurs at",
    "object": "colostomy site"
  },
  "fill_in_blank": "Carcinoma rarely occurs at a [BLANK].",
  "answer": "colostomy site",
  "distractors": [
    "ileostomy site",
    "rectal mucosa",
    "sigmoid colon"
  ],
  "question": "At which anatomical location does carcinoma infrequently develop?"
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Good — Replacing [BLANK] with “colostomy site” gives back the original, factually accurate sentence. It’s clear, grammatical, and self-contained; the subject (carcinoma) and location are specific enough for standalone understanding.",
    "paraphrase_quality": "Good — The question asks about where carcinoma infrequently develops, matching the meaning of “carcinoma rarely occurs at a colostomy site.” It’s natural English and understandable without the original sentence.",
    "options_quality": "Good — The answer matches exactly. Distractors are plausible anatomical sites related to colorectal surgery or carcinoma, similar in specificity and length, and not obviously absurd."
  },
  "fill_in_blank_quality": "Good",
  "paraphrase_quality": "Good",
  "options_quality": "Good"
}
```

### Input:
```json
{   
  "sentence": "Bronchoscopic resection of a bronchial tumor with an electric snare should be an initial approach of choice, because it is a quick and minimally invasive treatment, and enables thorough pathological evaluation.",
  "triple": {
    "subject": "Bronchoscopic resection of a bronchial tumor with an electric snare",
    "relation": "should be an initial approach of choice because it is",
    "object": "a quick and minimally invasive treatment that enables thorough pathological evaluation"
  },
  "fill_in_blank": "Bronchoscopic resection of a bronchial tumor with an electric snare should be an initial approach of choice, because it is [BLANK].",
  "question": "Why is bronchoscopic resection of a bronchial tumor with an electric snare recommended as the initial treatment approach?",
  "answer": "a quick and minimally invasive treatment that enables thorough pathological evaluation",
  "distractors": [
    "a lengthy and highly invasive procedure with limited diagnostic value",
    "an experimental therapy with uncertain outcomes and high complication rates",
    "a non-surgical intervention that does not allow for tissue analysis"
  ]
}
```

### Output: 
```json
{
  "explanations": {
    "fill_in_blank_quality": "Good — The fill-in-the-blank reproduces the original sentence when the object is inserted. It keeps the same meaning and is self-contained and grammatical.",
    "paraphrase_quality": "Good — The question restates exactly the same knowledge as the fill-in-the-blank, asking why the procedure is recommended; it’s natural and clear English.",
    "options_quality": "Poor — Although the correct answer matches, the distractors are clearly implausible and overly obvious (e.g., 'experimental therapy with uncertain outcomes' is extreme and unrealistic; 'non-surgical intervention' contradicts the bronchoscopic surgical context). They lack subtlety and don’t provide credible confusion."
  },
  "fill_in_blank_quality": "Good",
  "paraphrase_quality": "Good",
  "options_quality": "Poor"
}
```

### Input:
```json
{
  "sentences": "Its pharmacokinetic properties are superior to those of FLCZ, including a high penetration into the central nervous system.",
  "triple": {
    "subject": "the pharmacokinetic properties of this drug",
    "relation": "are superior to",
    "object": "those of FLCZ, including high penetration into the central nervous system"
  },
  "fill_in_blank": "[BLANK] are superior to those of FLCZ, including a high penetration into the central nervous system.",
  "answer": "the pharmacokinetic properties of this drug",
  "distractors": [
    "the pharmacodynamic properties of this drug",
    "the toxicity profile of this drug",
    "the metabolic stability of this drug"
  ],
  "question": "Which drug has pharmacokinetic properties that surpass those of FLCZ, particularly in terms of central nervous system penetration?"
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Poor — The fill-in-the-blank starts with an ambiguous pronoun “Its” in the original sentence and the object is ‘the pharmacokinetic properties of this drug’. Without naming the drug, the blanked sentence isn’t self-contained or unambiguous; a reader can’t know what “this drug” is. It’s not fully interpretable alone.",
    "paraphrase_quality": "Poor — The paraphrased question asks “Which drug has … superior pharmacokinetic properties?” but no drug name is given. It presupposes knowledge of what ‘this drug’ refers to; thus it isn’t self-contained and can’t be answered without outside context.",
    "options_quality": "Good — The correct answer exactly matches the fill-in-blank phrase. Distractors are in the right semantic field (pharmacodynamic, toxicity profile, metabolic stability), plausible alternatives, and similar style/length, making them credible without being obviously absurd."
  },
  "fill_in_blank_quality": "Poor",
  "paraphrase_quality": "Poor",
  "options_quality": "Good"
}
```

### Input:
{
  "sentence": "In this procedure, two separate teams perform the operation in the cervical and abdominal regions simultaneously, and thorough lymph node dissection is performed in each area.",
  "triple": {
    "subject": "two separate teams",
    "relation": "perform the operation simultaneously in",
    "object": "the cervical and abdominal regions"
  },
  "fill_in_blank": "In this procedure, [BLANK] perform the operation in the cervical and abdominal regions simultaneously, and thorough lymph node dissection is performed in each area.",
  "answer": "two separate teams",
  "distractors": [
    "a single surgical team",
    "three alternating teams",
    "two teams working sequentially"
  ],
  "question": "Who performs the operation simultaneously in both the cervical and abdominal regions during this procedure?"
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Poor — Although the blank can be filled grammatically, the sentence is just describing a procedural setup (two separate teams operating in two regions) and not conveying a clear biomedical fact about anatomy, pathology, or mechanism. It’s context dependent and not a stand-alone knowledge statement.",
    "paraphrase_quality": "Poor — The paraphrased question only asks who performs the operation but, like the fill-in-blank, it’s just procedural detail with no meaningful biomedical fact. It isn’t knowledge-based and would be unhelpful for fact recall tasks.",
    "options_quality": "Good — The correct answer matches the blank and distractors (single team, three alternating teams, two sequential teams) are plausible alternative surgical setups, making them balanced and credible."
  },
  "fill_in_blank_quality": "Poor",
  "paraphrase_quality": "Poor",
  "options_quality": "Good"
}
```

### Input:
```json
{
    "sentence": "These flaps are turned over and sutured to close the nasal floor, alveolar cleft and anterior palate cleft.",
    "triple": {
        "subject": "turned-over and sutured flaps",
        "relation": "are used to close",
        "object": "the nasal floor, alveolar cleft and anterior palate cleft"
    },
    "fill_in_blank": "These flaps are turned over and sutured to close the [BLANK].",
    "question": "Which anatomical regions are closed by turning over and suturing these flaps?",
    "answer": "nasal floor, alveolar cleft and anterior palate cleft",
    "distractors": [
        "soft palate, uvula and tonsillar fossa",
        "hard palate, maxillary sinus and zygomatic arch",
        "mandibular ramus, tongue and sublingual space"
    ]
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Poor — Although grammatically correct, the sentence begins with “These flaps” and never specifies what flaps are being discussed. Without prior context describing the flaps, the fact is not self-contained or interpretable on its own. A reader cannot know what structure or procedure is being referenced.",
    "paraphrase_quality": "Poor — The question similarly refers to “these flaps” without context, so it’s not self-contained and can’t be answered outside the original passage. It lacks clarity about what is being asked and depends on missing preceding information.",
    "options_quality": "Good — The correct answer matches the blank and the distractors are well-formed alternative anatomic regions; they are plausible and comparable in length and category."
  },
  "fill_in_blank_quality": "Poor",
  "paraphrase_quality": "Poor",
  "options_quality": "Good"
}
```


### Input:
```json
{
    "sentence": "Xylanase also catalyzed the transglycosylation in the presence of diphenol compounds and aromatic compounds that have both the phenolic and the alcoholic hydroxyl group.",
    "triple": {
        "subject": "Xylanase",
        "relation": "catalyzes",
        "object": "transglycosylation in the presence of diphenol compounds and aromatic compounds that have both phenolic and alcoholic hydroxyl groups"
    },
    "fill_in_blank": "Xylanase catalyzed the transglycosylation in the presence of [BLANK].",
    "question": "What types of compounds must be present for xylanase to catalyze transglycosylation?",
    "answer": "diphenol compounds and aromatic compounds that have both phenolic and alcoholic hydroxyl groups",
    "distractors": [
        "monophenol compounds and aliphatic compounds lacking hydroxyl groups",
        "aromatic compounds with only alcoholic hydroxyl groups and carboxylic acids",
        "diphenol compounds and aromatic compounds with only phenolic hydroxyl groups"
    ]
}
```

### Output:
```json
{
  "explanations": {
    "fill_in_blank_quality": "Good — Filling the blank with the object gives a sentence that is grammatically correct, factually faithful, and self-contained. It clearly states that xylanase catalyzes transglycosylation when certain phenolic/aromatic hydroxyl compounds are present. The subject (xylanase) and the chemical context are explicit and interpretable alone.",
    "paraphrase_quality": "Poor — The paraphrase uses “must be present,” which implies those compounds are strictly required for catalysis. The original sentence only observes activity in their presence, not that they are necessary. This semantic shift changes the fact from ‘observed with’ to ‘required for,’ so the question is not fully equivalent to the sentence.",
    "options_quality": "Good — The correct answer matches the blank, and the distractors are chemically plausible (similar compound types and functional groups) and balanced in length and complexity."
  },
  "fill_in_blank_quality": "Good",
  "paraphrase_quality": "Poor",
  "options_quality": "Good"
}
```
"""



import json
escaped_text = json.dumps({"system": text}, ensure_ascii=False, indent=4)
print(escaped_text)

{
    "system": "### Instruction\n\nYou are given a JSON input containing:\n- A **biomedical sentence**\n- An associated **knowledge triple** (`subject`, `relation`, `object`)\n- A **fill-in-the-blank query**\n- A **question-style paraphrase**\n\nYour task is to evaluate each component and return a JSON output with the specified structure.\n\n#### 1. Fill-in-the-Blank Query Evaluation\nA fill-in-the-blank query is **Good** only if **all** conditions hold:\n- **Faithful reconstruction:** Inserting the `object` into `[BLANK]` reproduces a sentence that keeps the original **meaning and factual correctness**.\n- **Contextual completeness:** \n  - The query must contain enough information to answer correctly from the sentence **without needing extra context**. If key qualifiers are missing and make the question ambiguous, mark Poor.\n  - Avoid overly broad or context-dependent claims that require knowing the surrounding text to interpret.\n- **Syntactic & semantic clarity:** The prompt must

In [2]:
### Cross-lingual filtering
text = """\" \
### Instruction:

You are given a JSON input as follows:
- **Original Sentence (English)**: the sentence conveys the biomedical fact we want to check.
- **Knowledge Triple**: a structured representation of the fact in the form (subject, relation, object).
- **Related Abstract (Japanese)**: a Japanese abstract that may provide context for the sentence.
- **Question**: a question derived from the original sentence.
- **Answer**: the answer to the question.

Your task is to score how well the **Related Abstract** alone allows a bilingual reader to recover the same Question/Answer pair derived from the **Original Sentence**.

### Scoring Scale: 
**2 — Fully Supported:** 
- The Japanese abstract contains one self-contained sentence (or one clearly bounded pair of consecutive sentences) that already expresses the entire English fact — the same subject, relation, object, and any key qualifiers.
- All key elements (subject, relation, object, qualifiers) appear explicitly in the Japanese text and can be matched to the English fact without guessing or outside knowledge.
- The support must come from a self-contained Japanese sentence or clearly bounded set of sentences where the subject is explicitly named (no pronouns if the referent isn’t in the same evidence).
- The Japanese wording can be matched almost one-to-one with the English sentence without reassembling information from multiple parts.
- The bilingual reader can map the English fact to Japanese wording almost one-to-one.
- Strict evaluation: if there is any doubt about whether the abstract alone is enough to fully support the Q/A pair, assign a lower score.

**1 — Partially Supported**: The abstract provides some relevant evidence but is incomplete or ambiguous:
- The Japanese abstract gives enough clues to reasonably deduce the Q/A but the fact is not stated in a single direct sentence.
- Some pieces might be implicit or require linking nearby sentences (e.g., subject in one, property in next), but no external biomedical expertise is required — just reading comprehension and bilingual ability.
- A bilingual reader might guess the correct answer but with some uncertainty.
- Strict evaluation: If the reasoning depends on medical or domain knowledge beyond the text, it must not be scored 1 (should be 0 instead).

**0 — Not Supported**: 
- The abstract does not provide enough information to answer the question.
- The fact or answer is absent, too vague, or would require external biomedical/domain knowledge to infer.
- Any case where a bilingual but non-expert reader cannot get the Q/A from the abstract alone.

### Strict Evaluation Criteria:
- Be conservative. If the Japanese abstract is not a direct factual match, or if any part of the Q/A can only be guessed by applying external biomedical knowledge, assign 0.
- Only award 2 when you can point to a direct Japanese sentence(s) clearly expressing the entire fact.

### Input Format (JSON):
```json
{
    "ja-context": "<related abstract in Japanese>",
    "en-sentence": "<original sentence in English>",
    "en-triple": {
        "subject": "<subject>",
        "relation": "<relation>",
        "object": "<object>"
    },
    "en-question": "<question derived from the sentence>",
    "en-answer": "<answer to the question>"
}
```

### Output Format (JSON):
```json
{
    "explanation": "<brief justification for the score>",
    "associated_sentences": ["<list of sentences from the Japanese abstract that are related to the Q/A>"],
    "score": 0 | 1 | 2,
}
```

### Input:
```json
{
    "ja-context": "血栓回収療法の登場により脳梗塞治療はいま大きく変化している．脳梗塞に対する血管内治療を普及させるためのプロジェクト（RESCUE-Japan Project）による全国調査では，2016 年に7702 例（6.06 件/10 万人/年）の血栓回収療法が行われたが，これは全脳梗塞の約6％と試算される．我が国での血栓回収療法の普及は急務であり，治療体制の整備が求められている．一方で，血栓回収療法が適応となっても半数の患者は機能的自立が得られない．後遺症が残存した場合の治療として，我々は内因性の神経幹細胞である傷害誘導性多能性幹細胞（ischemia-induced multipotent stem cells: iSCs）を用いた神経再生療法に期待している．iSCs は脳梗塞巣内に存在し，良好な自己増殖能を持つ．その起源はペリサイトと考えられ，神経堤の性格をも併せ持ち，神経へ分化可能である．iSCs は生体の持つ神経修復機構に関与していると考えられ，近い将来の臨床応用が期待される．",
    "en-sentence": "Such cells have characteristics of the neural crest, thereby indicating that they are able to differentiate into neurons.",
    "en-triple": {
        "subject": "cells with characteristics of the neural crest",
        "relation": "are able to differentiate into",
        "object": "neurons"
    },
    "en-question": "Cells exhibiting neural crest characteristics have the ability to develop into which type of cell?",
    "en-answer": "neurons"
}
```

### Output:
```json
{
    "explanation": "The Japanese abstract explicitly states that iSCs (ischemia-induced multipotent stem cells) have neural crest characteristics (神経堤の性格をも併せ持ち) and that they are capable of differentiating into neurons (神経へ分化可能である). This directly matches the English sentence and triple: cells with neural crest traits can differentiate into neurons. Thus, a bilingual biomedical reader could confidently answer the question using the abstract alone.",
    "associated_sentences": [
        "iSCs は脳梗塞巣内に存在し，良好な自己増殖能を持つ．その起源はペリサイトと考えられ，神経堤の性格をも併せ持ち，神経へ分化可能である．"
    ],
    "score": 2
}
```

### Input:
```json
{
    "ja-context": "抗β-lactamase性とグラム陽性菌からグラム陰性菌に及ぶ幅広い抗菌スペクトルを有し, さらにSerratiaおよびPseudomonas aeruginosaeに対して強い抗菌力を有する新しいcephalosporin系抗生剤Ceftazidime (CAZ, SN401) を外来4例・入院19例, 11歳より85歳, 男性9例。女性14例の急性虫垂炎に合併せる限局性腹膜炎10例, クローン氏病あるいは腸閉塞による汎発性腹膜炎3例, 胆石症による胆道系感染症5例 (6疾患), 痔瘻1例, 外来の皮膚軟部組織感染症4例に使用し, 著効8例, 有効l6例の成績を得た。全例において副作用および本剤によると思われる臨床検査値の異常は認められなかった。起炎菌の判明したものは17例で, うち7例は複数菌感染であった。菌種別ではEscherichia coli16株, Klebsiella Pneumeniae 4株, Bacteroides fragais2株, Pseudomonas aeruginosa 2株, Enterobacter cloacae1株, 計25株であった。大多数を占める大腸菌のMICは108, 106cells/mlにて0.39, 0.2μg/mlであった。症例のうち急性腹膜炎の11例, 胆石症の5例, 痔瘻1例の計17例の手術時にCAZ1gを静注し, 各種体液, 組織内の濃度を測定した0測定法はProteus mirabilis ATCC 21100を検定菌とするbioassay法である。膿性腹水, 胆嚢壁, 虫垂壁, 痔瘻などの炎症病巣への本剤の移行は極めて良好で, 起炎菌のMICを大幅に上回っており, 本剤の有用性が認められた。本剤は皮膚・軟部組織, 胆道および腹部感染症に対して極めて有用な薬剤であるといえる。",
    "en-sentence": "From the above results, ceftazidime is considered to be a useful drug for the therapy of skin and soft tissue infections, biliary tract and intra-abdominal infections.",
    "en-triple": {
        "subject": "ceftazidime",
        "relation": "is considered to be a useful drug for the therapy of",
        "object": "skin and soft tissue infections, biliary tract infections, and intra-abdominal infections"
    },
    "en-question": "For which types of infections is ceftazidime regarded as a useful therapeutic drug?",
    "en-answer": "skin and soft tissue infections, biliary tract infections, and intra-abdominal infections"
}
```

### Output:
```json
{
    "explanation": "The Japanese sentence states that the drug is extremely useful for skin and soft tissue, biliary tract, and intra-abdominal infections, but it refers to the drug only as '本剤' (this agent) without naming ceftazidime in that sentence. Identifying that '本剤' means ceftazidime depends on earlier context, so the fact cannot be fully recovered from this sentence alone. Therefore, the support is only partial.",
    "associated_sentences": [
        "本剤は皮膚・軟部組織, 胆道および腹部感染症に対して極めて有用な薬剤であるといえる。"
    ],
    "score": 1
}
```

### Input:
```json
{
    "ja-context": "水中の22種農薬における多成分分析法として, 自動固相抽出装置とLC-APCI/MSを組み合わせた方法を確立した. 試料水 (精製水および河川水) は, 自動固相抽出装置を用いてポリマー系カートリッジ (Sep-pak PS-2 plus cartridge) に濃縮後, 抽出液はLC-APCI/MSを用いてSIM定量した. 今回, 22種の農薬における検出限界値は0.02～0.3μg/lであり, 精製水と河川水の10μg/lから30μg/lにおける添加回収試験は, エトフェンプロックスを除いた21農薬において, それぞれ81～103%と71～127%であった. GCもしくはHPLC等では分析が困難とされる極性化合物のアシュラム, カルベンダジムやスルホニル尿素系除草剤, もしくは, 熱不安定でUV吸収の少ないトリクロルホン等も, 誘導体化することなく測定でき, さらに, MSを用いていることで選択性の高い, 環境水における微量分析法として簡便な測定方法となった.",
    "en-sentence": "In general, polar or thermal labile pesticides such as asulam, benzimidazole, trichlorfon and sulfonylureas were necessary to be analyzed by gas chromatographic method after derivertization.",
    "en-triple": {
        "subject": "polar or thermally labile pesticides such as asulam, benzimidazole, trichlorfon and sulfonylureas",
        "relation": "require analysis by",
        "object": "gas chromatographic method after derivatization"
    },
    "en-question": "Which types of pesticides typically require analysis by gas chromatographic method after derivatization?",
    "en-answer": "polar or thermally labile pesticides such as asulam, benzimidazole, trichlorfon and sulfonylureas"
}
```

### Output:
```json
{
    "explanation": "The Japanese abstract explains that a new LC-APCI/MS method was established for 22 pesticides and specifically notes that certain polar, thermally unstable compounds such as asulam, carbendazim (a benzimidazole), sulfonylurea herbicides, and trichlorfon can be measured without derivatization, whereas GC or HPLC is generally difficult for them. This implies that these compounds normally require derivatization for GC analysis. The main pesticide classes (polar, thermally labile, asulam, benzimidazole, trichlorfon, sulfonylureas) are explicitly mentioned, but the fact that they ‘require gas chromatographic method after derivatization’ is only implied by contrast (they are hard to analyze by GC/HPLC). A bilingual reader could infer but not with full certainty, so the support is partial.",
    "associated_sentences": [
        "GCもしくはHPLC等では分析が困難とされる極性化合物のアシュラム, カルベンダジムやスルホニル尿素系除草剤, もしくは, 熱不安定でUV吸収の少ないトリクロルホン等も, 誘導体化することなく測定でき, さらに, MSを用いていることで選択性の高い, 環境水における微量分析法として簡便な測定方法となった."
    ],
    "score": 1
}
```

### Input:
```json
{
    "ja-context": "ブドウ‘巨峰’の菌根について, 形態的特徴や菌の侵入及びその季節的変動を調べた. また, 罹病根との比較も行った. 菌根は新しく形成された根に観察され, 典型的なVA菌根であった. 嚢状体は径20～70μmで卵形及び球形のものが多かったが, なかには不規則な形のものも観察された. 分岐体は袋状の膜に包まれていた. 嚢状体は10月から12月の根に, 分岐体は7月から8月の根において多く形成された. VA菌根菌は自家蛍光を持っており,蛍光色素を用いないで蛍光顕微鏡で侵入が観察できた.菌の侵入量は10月以降に増加する傾向がみられた. まれに Rhizoctonia 様内生菌も観察された. 罹病根における病原菌の侵入に比しVA菌根菌の根への侵入は根の内部にまで多くの菌糸が侵入していたが, 細胞死や枯死などは起きておらず明らかに罹病根とは様相が異なっていた.",
    "en-sentence": "Pathogenic fungi (Rosellinia necatrix and Helicobacidium mompa) killed the host cell and host plant by attacking only epidermal cells, whereas VA mycorrhizal fungi penetrated into the inner root tissues without causing cell or plant death.",
    "en-triple": {
        "subject": "Pathogenic fungi Rosellinia necatrix and Helicobacidium mompa",
        "relation": "kill by attacking only",
        "object": "epidermal cells of the host plant"
    },
    "en-question": "Which specific cells do the pathogenic fungi Rosellinia necatrix and Helicobacidium mompa attack to kill the host plant?",
    "en-answer": "epidermal cells of the host plant"
}
```

### Output:
```json
{
    "explanation": "The Japanese abstract only mentions that diseased roots have pathogenic fungal invasion unlike VA mycorrhizal fungi but gives no details on the specific fungi (Rosellinia necatrix, Helicobacidium mompa), no information that they kill the host, and no statement that they attack epidermal cells. There is not enough explicit or implied information to answer the question or identify the correct answer.",
    "associated_sentences": [
        "罹病根における病原菌の侵入に比しVA菌根菌の根への侵入は根の内部にまで多くの菌糸が侵入していたが, 細胞死や枯死などは起きておらず明らかに罹病根とは様相が異なっていた."
    ],
    "score": 0
}
```

### Input:
```json
{
    "ja-context": "頭蓋外内血行再建術は脳虚血症状を有するもやもや病に対する有効な治療法である. Japan Adult Moyamoya trialの結果を受け, 直接血行再建術を含めた血行再建術の適応は成人出血発症例にも近年拡大傾向にある. 中でも後方循環系の出血例に対しては血行再建術による再出血予防効果が高く, 手術が特に勧められる. JAM trial後の当該施設における成人出血発症例に対する直接間接複合手術の結果も良好であり, 成人出血発症例に対する血行再建術は有効な安全, 有効な治療法と考えられた. 周術期においては脳虚血, 過灌流症候群の回避が重要である.",
    "en-sentence": "Moyamoya disease is a chronic cerebrovascular disease with unknown etiology characterized by steno-occlusive changes at the terminal portion of the internal carotid artery and an abnormal vascular network at the base of the brain.",
    "en-triple": {
        "subject": "Moyamoya disease",
        "relation": "is characterized by",
        "object": "steno-occlusive changes at the terminal portion of the internal carotid artery and an abnormal vascular network at the base of the brain"
    },
    "en-question": "What are the characteristic vascular changes seen in Moyamoya disease?",
    "en-answer": "steno-occlusive changes at the terminal portion of the internal carotid artery and an abnormal vascular network at the base of the brain"
}
```

### Output:
```json
{
    "explanation": "The Japanese abstract discusses treatments for Moyamoya disease and mentions surgical revascularization as effective, but it does not describe the fundamental vascular pathology of the disease. There is no mention of steno-occlusive changes at the terminal internal carotid artery or abnormal vascular networks at the brain base. Thus, the key diagnostic vascular features needed to answer the question are missing.",
    "associated_sentences": [
        "頭蓋外内血行再建術は脳虚血症状を有するもやもや病に対する有効な治療法である.",
        "Japan Adult Moyamoya trialの結果を受け, 直接血行再建術を含めた血行再建術の適応は成人出血発症例にも近年拡大傾向にある."
    ],
    "score": 0
}
```

### Input:
```json
{
    "ja-context": "血友病患者にヒト第VIII因子および第IX因子製剤を輸注することにより発生する。抗第VIII因子および第IX因子同種抗体は，止血管理上重要な問題である。今回私たちは血友病インヒビター症例における遺伝的および免疫生化学的特徴につき検討した。血友病A（8例），およびB（6例）のインヒビター症例において，第VIII因子遺伝子についてはサザンブロット法により，また第IX因子遺伝子についてはサザンブロット法およびシークエンスにより遺伝子解析を行った。その結果，血友病A4例，B3例に大きな遺伝子欠損を認めた。また患者血清を用いて各種第VIII因子製剤および第IX因子製剤に対する認識部位の解析を行ったところ，各種製剤に対する認識パターンは製剤により異なっており，製剤の性状の違いによるものと思われた。また血友病Aインヒビター12例中11例がPorcine第VIII因子にも反応していた。",
    "en-sentence": "In some literature, about 60% of hemophila A inhibitor patients have shown gross gene deletions.",
    "en-triple": {
        "subject": "about 60% of hemophilia A inhibitor patients",
        "relation": "have been reported to show",
        "object": "gross gene deletions"
    },
    "en-question": "What genetic abnormality has been reported in approximately 60% of hemophilia A inhibitor patients according to some literature?",
    "en-answer": "gross gene deletions"
}
```

"""

import json
escaped_text = json.dumps({"system": text}, ensure_ascii=False, indent=4)
print(escaped_text)

{
    "system": "\" ### Instruction:\n\nYou are given a JSON input as follows:\n- **Original Sentence (English)**: the sentence conveys the biomedical fact we want to check.\n- **Knowledge Triple**: a structured representation of the fact in the form (subject, relation, object).\n- **Related Abstract (Japanese)**: a Japanese abstract that may provide context for the sentence.\n- **Question**: a question derived from the original sentence.\n- **Answer**: the answer to the question.\n\nYour task is to score how well the **Related Abstract** alone allows a bilingual reader to recover the same Question/Answer pair derived from the **Original Sentence**.\n\n### Scoring Scale: \n**2 — Fully Supported:** \n- The Japanese abstract contains one self-contained sentence (or one clearly bounded pair of consecutive sentences) that already expresses the entire English fact — the same subject, relation, object, and any key qualifiers.\n- All key elements (subject, relation, object, qualifiers) appear